<a href="https://colab.research.google.com/github/AsanAlacli/JSLABS/blob/main/keyModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import json

from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

Saving 3.0.0. OCR_spark_nlp_for_healthcare.txt to 3.0.0. OCR_spark_nlp_for_healthcare.txt


In [3]:
%%capture
for k,v in license_keys.items(): 
    %set_env $k=$v

!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jsl_colab_setup.sh
!bash jsl_colab_setup.sh

! pip install spark-nlp-display

In [4]:
import json
import os
from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql import SparkSession

from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl
import sparknlp

params = {"spark.driver.memory":"16G",
"spark.kryoserializer.buffer.max":"2000M",
"spark.driver.maxResultSize":"2000M"}

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print ("Spark NLP Version :", sparknlp.version())
print ("Spark NLP_JSL Version :", sparknlp_jsl.version())

Spark NLP Version : 3.0.1
Spark NLP_JSL Version : 3.0.0


In [61]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP
documentAssembler = DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

'sentenceDetector = SentenceDetectorDLModel.pretrained("sentence_detector_dl_healthcare","en","clinical/models")\
        .setInputCols(["document"])\
        .setOutputCol("sentence")'

'tokenizer = sparknlp.annotators.Tokenizer()\
    .setInputCols(["sentences"])\
    .setOutputCol("tokens")'

nlpPipeline = Pipeline(stages=[
        documentAssembler
        #sentenceDetector,
        #tokenizer
        ])


empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

In [62]:
model.stages

[DocumentAssembler_3e2127ce0ae8]

In [63]:
import pyspark.sql.functions as F

first_spark_df = spark.read\
                .option("header", "true")\
                .csv("mimic_100_pats.csv")\
                
first_spark_df.show(truncate=770)

+----------------------------------------------------------------------------+------+--------------------+--------+----------+-------------------------------------------------------------------------------+
|                                                                         _c0|ROW_ID|          SUBJECT_ID| HADM_ID| CHARTDATE|                                                                           TEXT|
+----------------------------------------------------------------------------+------+--------------------+--------+----------+-------------------------------------------------------------------------------+
|                                                                         744|   934|               18589|181162.0|2191-05-10|Admission Date:  [**2191-4-18**]              Discharge Date:   [**2191-5-10**]|
|                        Date of Birth:  [**2112-3-14**]             Sex:   M|  null|                null|    null|      null|                                              

In [114]:
#first_spark_df.show()
text = first_spark_df.select('_c0').take(1)[0][0]
text

'744'

In [58]:
lmodel = LightPipeline(model)

results = lmodel.fullAnnotate(text)

In [59]:
results[0].keys()

dict_keys(['document'])

In [60]:
results[0]

{'document': [Annotation(document, 0, 2, 744, {})]}

In [65]:
import pandas as pd

In [68]:
pandas_df=pd.read_csv('mimic_100_pats.csv')
pandas_df

,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,TEXT
0,744,934,18589,181162.0,2191-05-10,Admission Date: [**2191-4-18**] ...
1,1315,1125,17581,144831.0,2126-05-22,Admission Date: [**2126-5-7**] Discharge ...
2,1316,1126,17581,140044.0,2127-07-29,Admission Date: [**2127-7-27**] ...
3,2117,1896,8904,147403.0,2108-03-13,Admission Date: [**2108-2-22**] ...
4,2761,2872,18054,138511.0,2147-12-11,Admission Date: [**2147-11-22**] Discharg...
...,...,...,...,...,...,...
9637,2080622,2054683,29266,102339.0,2128-07-03,Neonatology\r\nDoing well. remains in NCO2 flo...
9638,2080623,2054684,29266,102339.0,2128-07-03,Neonatology NP Note\r\nPE\r\nswaddled in open ...
9639,2080624,2054685,29266,102339.0,2128-07-03,Nursing NICU Note\r\n\r\n\r\n#1. Respiratory ...
9640,2080720,2054641,29266,102339.0,2128-06-24,Neonatology Attending\r\nDOL 3 / PMA 40-4/7 we...


In [73]:
pandas_df.columns[0]

'Unnamed: 0'

In [75]:
pandas_df = pandas_df.rename(columns={'Unnamed: 0' : 'UKNWN_COL'})

In [77]:
pandas_df.head(20)

,UKNWN_COL,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,TEXT
0,744,934,18589,181162.0,2191-05-10,Admission Date: [**2191-4-18**] ...
1,1315,1125,17581,144831.0,2126-05-22,Admission Date: [**2126-5-7**] Discharge ...
2,1316,1126,17581,140044.0,2127-07-29,Admission Date: [**2127-7-27**] ...
3,2117,1896,8904,147403.0,2108-03-13,Admission Date: [**2108-2-22**] ...
4,2761,2872,18054,138511.0,2147-12-11,Admission Date: [**2147-11-22**] Discharg...
5,2762,2873,18054,191467.0,2157-12-02,Admission Date: [**2157-11-25**] ...
6,3414,3704,52574,109934.0,2180-08-31,Admission Date: [**2180-8-28**] ...
7,3415,3705,52574,189995.0,2180-10-29,Admission Date: [**2180-10-19**] ...
8,3416,3706,52574,177706.0,2181-02-27,Admission Date: [**2181-2-23**] ...
9,4287,4273,11425,194366.0,2100-09-24,Admission Date: [**2100-9-5**] Discharg...


In [78]:
pandas_df.tail(20)

,UKNWN_COL,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,TEXT
9622,2080058,2054625,29266,102339.0,2128-06-22,NPN\r\n\r\n\r\n#1RESP: Recv'd infant in NC 1 l...
9623,2080059,2054626,29266,102339.0,2128-06-22,Respiratory Care\r\nInfant placed in oxyhood t...
9624,2080060,2054627,29266,102339.0,2128-06-22,Neonatology Attending\r\nDOL 1 / PMA 40-2/7 we...
9625,2080061,2054686,29266,102339.0,2128-07-04,NICU NPN 1900-0700\r\n\r\n\r\nRESP O: [**Known...
9626,2080181,2054650,29266,102339.0,2128-06-26,npn 1900-0700\r\n\r\n\r\n#1 resp\r\ninfant con...
9627,2080182,2054651,29266,102339.0,2128-06-26,"Neonatology Attending Note\r\nDay 5, PMA 40 6\..."
9628,2080183,2054652,29266,102339.0,2128-06-26,NPN DAYS\r\n\r\n\r\nALT IN RESP:REMAINS IN NAS...
9629,2080184,2054714,29266,102339.0,2128-07-10,Attending Note\r\nDay of life 19 PMA 42 [**5-1...
9630,2080330,2054622,29266,102339.0,2128-06-21,Neonatology Attending\r\nFull term infant admi...
9631,2080331,2054623,29266,102339.0,2128-06-21,Neonatology Attending\r\n(Continued)\r\nymptom...


In [83]:
pandas_df.shape

(9642, 6)

In [84]:
pandas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9642 entries, 0 to 9641
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   UKNWN_COL   9642 non-null   int64  
 1   ROW_ID      9642 non-null   int64  
 2   SUBJECT_ID  9642 non-null   int64  
 3   HADM_ID     8222 non-null   float64
 4   CHARTDATE   9642 non-null   object 
 5   TEXT        9642 non-null   object 
dtypes: float64(1), int64(3), object(2)
memory usage: 452.1+ KB


In [86]:
pandas_df.describe()

,UKNWN_COL,ROW_ID,SUBJECT_ID,HADM_ID
count,9.642000e+03,9.642000e+03,9642.000000,8222.000000
mean,1.098950e+06,1.099887e+06,27687.871292,151963.337509
std,5.859361e+05,5.863771e+05,24335.446171,29731.451320
min,7.440000e+02,9.340000e+02,403.000000,100481.000000
25%,6.544888e+05,6.555862e+05,11425.000000,128942.000000
50%,1.074476e+06,1.069898e+06,19823.000000,147403.000000
75%,1.622852e+06,1.625378e+06,30015.000000,177882.000000
max,2.080721e+06,2.054715e+06,99647.000000,199558.000000


In [88]:
pandas_df.sample(n=8)

,UKNWN_COL,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,TEXT
1632,407149,405350,29810,197536.0,2106-04-13,History of metastatic renal cell carcinoma und...
7665,1699316,1735154,4186,118086.0,2150-08-26,NNP Procedure Note\r\nCT removal\r\n\r\nLeft C...
303,90115,89660,42715,146156.0,2165-01-24,PATIENT/TEST INFORMATION:\r\nIndication: Endoc...
4519,1016833,1018883,28552,183345.0,2122-07-20,"[**Last Name (LF) **],[**First Name3 (LF) 3675..."
204,64397,64172,52574,NaN,2180-08-07,PATIENT/TEST INFORMATION:\r\nIndication: Aorti...
5330,1180722,1183971,52828,NaN,2133-03-15,[**2133-3-15**] 9:36 PM\r\n CTA ABD W&W/O C & ...
3464,817038,817888,1401,NaN,2179-03-03,[**2179-3-3**] 7:44 PM\r\n MR SHOULDER W/O CON...
1364,359205,361973,698,171990.0,2167-12-23,89 year old patient admitted from the EW with ...


In [93]:
pandas_df.isna().any()

UKNWN_COL     False
ROW_ID        False
SUBJECT_ID    False
HADM_ID        True
CHARTDATE     False
TEXT          False
dtype: bool

In [92]:
pandas_df.isnull().sum()

UKNWN_COL        0
ROW_ID           0
SUBJECT_ID       0
HADM_ID       1420
CHARTDATE        0
TEXT             0
dtype: int64

In [111]:
text = pandas_df.TEXT[0]

In [112]:
text

'Admission Date:  [**2191-4-18**]              Discharge Date:   [**2191-5-10**]\r\n\r\nDate of Birth:  [**2112-3-14**]             Sex:   M\r\n\r\nService: MEDICINE\r\n\r\nAllergies:\r\nCeftriaxone\r\n\r\nAttending:[**First Name3 (LF) 2297**]\r\nChief Complaint:\r\nPneumonia\r\n\r\nMajor Surgical or Invasive Procedure:\r\nnone\r\n\r\nHistory of Present Illness:\r\nThis is a 79 yo recently diagnosed with [**Doctor Last Name 6261**] syndrome\r\n(transformation of CLL to large B cell NHL) now s/p 3 cycles of\r\nCHOP presents with PNA diagnosed at [**Hospital1 **] ED.  The pt is\r\nnow referred from [**Hospital 478**] clinic for treatment of PNA.  The pt\r\npresented to OSH ED on [**4-13**] and was diagnosed with LLL PNA and\r\nwas given a course of levofloxacin.  The pt did not take his abx\r\nas prescribed.  He c/o increased SOB, DOE, and cough x 1 week.\r\nThe pt also c/o decrease appetite over past week with 4 lb\r\nweight loss.  No increase in fatigue.  No f/c/s, n/v, d/c,\r\nabdomin

In [113]:
sparkDF= spark.createDataFrame(pandas_df)
sparkDF.show()

+---------+------+----------+--------+----------+--------------------+
|UKNWN_COL|ROW_ID|SUBJECT_ID| HADM_ID| CHARTDATE|                TEXT|
+---------+------+----------+--------+----------+--------------------+
|      744|   934|     18589|181162.0|2191-05-10|Admission Date:  ...|
|     1315|  1125|     17581|144831.0|2126-05-22|Admission Date:  ...|
|     1316|  1126|     17581|140044.0|2127-07-29|Admission Date:  ...|
|     2117|  1896|      8904|147403.0|2108-03-13|Admission Date:  ...|
|     2761|  2872|     18054|138511.0|2147-12-11|Admission Date:  ...|
|     2762|  2873|     18054|191467.0|2157-12-02|Admission Date:  ...|
|     3414|  3704|     52574|109934.0|2180-08-31|Admission Date:  ...|
|     3415|  3705|     52574|189995.0|2180-10-29|Admission Date:  ...|
|     3416|  3706|     52574|177706.0|2181-02-27|Admission Date:  ...|
|     4287|  4273|     11425|194366.0|2100-09-24|Admission Date:  ...|
|     4998|  4994|     19823|179754.0|2167-02-25|Admission Date:  ...|
|     

In [115]:
lmodel = LightPipeline(model)

results = lmodel.fullAnnotate(text)

In [116]:
results[0].keys()

dict_keys(['document'])

In [117]:
results[0]

{'document': [Annotation(document, 0, 2, 744, {})]}